USING MEDIAPIPE

In [120]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import numpy as np

PATH = 'REC7850246504596738253.mp4'
 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
def start_here(path):
  cap = cv2.VideoCapture(path)
  
  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose: 
  # Read until video is completed
    while(cap.isOpened()):
      # Capture frame-by-frame
      ret, frame = cap.read()
      if ret == True:
          image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          image.flags.writeable = False
        
          # Make detection
          results = pose.process(image)
      
          # Recolor back to BGR
          image.flags.writeable = True
          image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
          
          try:
              landmarks = results.pose_landmarks.landmark
          except:
              pass
            
          mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                      mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2),
                                      )
          
        # Display the resulting frame
          cv2.imshow('Frame',image)
    
        # Press Q on keyboard to  exit
          if cv2.waitKey(25) & 0xFF == ord('q'):
              break
    
      # Break the loop
      else: 
        break
  
  # When everything done, release the video capture object
  cap.release()
  
  # Closes all the frames
  cv2.destroyAllWindows()
  return results

results = start_here(PATH)

In [121]:
len(results.pose_landmarks.landmark)

33

In [122]:
dict_landmarks = {
    0 : "nose",
    1 : "left_eye(inner)",
    2 : "left_eye",
    3 : "left_eye(outer)",
    4 : "right_eye(inner)",
    5 : "right_eye",
    6 : "right_eye(outer)",
    7 : "left_ear",
    8 : "right_ear",
    9 : "mouth(left)",
    10 : "mouth(right)",
    11 : "left_shoulder",
    12 : "right_shoulder",
    13 : "left_elbow",
    14 : "right_elbow",
    15 : "left_wrist",
    16 : "right_wrist",
    17 : "left_pinky",
    18 : "right_pinky",
    19 : "left_index",
    20 : "right_index",
    21 : "left_thumb",
    22 : "right_thumb",
    23 : "left_hip",
    24 : "right_hip",
    25 : "left_knee",
    26 : "right_knee",
    27 : "left_ankle",
    28 : "right_ankle",
    29 : "left_heel",
    30 : "right_heel",
    31 : "left_foot_index",
    32 : "right_foot_index"
}

In [123]:
import csv
import pandas as pd

def create_csv(name_of_file: str):
    num_coords = len(results.pose_landmarks.landmark)
    landmarks = []
    
    for val in range(0, num_coords):
        landmarks += [
            'x-{}'.format(dict_landmarks[val]), 
            'y-{}'.format(dict_landmarks[val]), 
            ]
    with open(name_of_file, mode='w', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(landmarks)
        
    df = pd.read_csv(name_of_file)
    new_df = pd.concat([df,pd.DataFrame(columns=['x_mid_point_sh',
            'y_mid_point_sh',
            'x_nor',
            'y_nor',
            'angel'])],axis='columns')
    new_df.to_csv(name_of_file,index=False)
    
create_csv('coords.csv')

FOR DETECTION AND COORDINATES

In [124]:
# calculate angel

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))


def execute_angle(pose_row):
    
    x_right_shoulder,y_right_shoulder = pose_row[24],pose_row[25]
    x_left_shoulder,y_left_shoulder = pose_row[22],pose_row[23]
    k=-0.4
    
    x_right_ear,y_right_ear = pose_row[16],pose_row[17]
    x_left_ear,y_left_ear = pose_row[14],pose_row[15]
    x_mid_point = (x_right_shoulder + x_left_shoulder)/2
    y_mid_point = (y_right_shoulder + y_left_shoulder)/2
    x_nor_vector = y_right_shoulder - y_left_shoulder
    y_nor_vector = x_left_shoulder - x_right_shoulder
    
    x_nor = x_mid_point + k*x_nor_vector+0.02
    y_nor = y_mid_point + k*y_nor_vector+0.02

    angle = angle_between((x_nor,y_nor),(x_left_ear,y_left_ear))
    
    return [x_mid_point,y_mid_point,x_nor,y_nor,angle]


In [125]:
def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

def findAngle(x1, y1, x2, y2):
    theta = m.acos( (y2 -y1)*(-y1) / (m.sqrt(
        (x2 - x1)**2 + (y2 - y1)**2 ) * y1) )
    degree = int(180/m.pi)*theta
    return degree

In [126]:
import cv2
import mediapipe as mp
import csv
import os
import math as m
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
import numpy as np
from PIL import Image

def findDistance(x1, y1, x2, y2):
    dist = m.sqrt((x2-x1)**2+(y2-y1)**2)
    return dist

def findAngle(x1, y1, x2, y2):
    theta = m.acos( (y2 -y1)*(-y1) / (m.sqrt(
        (x2 - x1)**2 + (y2 - y1)**2 ) * y1) )
    degree = int(180/m.pi)*theta
    return degree
  
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
def get_keypoints(path):
  
  count = 0
  good_frames = 0
  bad_frames  = 0
  percent = 20
  # Colors.
  blue = (255, 127, 0)
  red = (50, 50, 255)
  green = (127, 255, 0)
  dark_blue = (127, 20, 0)
  light_green = (127, 233, 100)
  yellow = (0, 255, 255)
  pink = (255, 0, 255)
  cap = cv2.VideoCapture(path)
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose()
  font = cv2.FONT_HERSHEY_SIMPLEX
  fps = int(cap.get(cv2.CAP_PROP_FPS))
  width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

  frame_size = (width, height)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    
  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  images = []

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
  # Read until video is completed
    while(cap.isOpened()):
      
      ret, frame = cap.read()
      if ret == True:
        
          width = int(frame.shape[1] * 20 / 40) 
          height = int(frame.shape[0] * 20 / 40) 
          dim = (width, height) 
          frame = cv2.resize(frame, dim)
          # Recolor Feed
          image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          image.flags.writeable = False  
          h, w = image.shape[:2]     
          # Make Detections
          results = holistic.process(image)
          try:
            
              pose = results.pose_landmarks.landmark
              pose_row = list(np.array([[landmark.x, landmark.y] for landmark in pose]).flatten())
              pose_row.extend(execute_angle(pose_row))
              
              #Export to CSV
              with open('coords.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row) 
              
          except:
              pass
          # draw line from middle of the shoulders an right ear
          color = 'default'
          angle = pose_row[-1]
          
          cv2.circle(image, (int(pose_row[-5]*w), int(pose_row[-4]*h)), 6, (255,255,255), -1) 
          cv2.circle(image, (int(pose_row[-3]*w), int(pose_row[-2]*h)), 4,  (255,255,255), -1)
            
          if pose_row[-1]>0.048 and pose_row[-1]< 0.14:
              cv2.line(image, (int(pose_row[-5]*w), int(pose_row[-4]*h)), (int(pose_row[16]*w), int(pose_row[17]*h)), red, 2)
          
              cv2.line(image, (int(pose_row[-3]*w), int(pose_row[-2]*h)), (int(pose_row[-5]*w), int(pose_row[-4]*h)), red, 2)
              
          else:
              
              cv2.line(image, (int(pose_row[-5]*w), int(pose_row[-4]*h)), (int(pose_row[16]*w), int(pose_row[17]*h)), (255, 255, 255), 2)
              
              cv2.line(image, (int(pose_row[-3]*w), int(pose_row[-2]*h)), (int(pose_row[-5]*w), int(pose_row[-4]*h)), (255, 255, 255), 2)
          
          lm = results.pose_landmarks
          lmPose  = mp_pose.PoseLandmark
          # Left shoulder.
          l_shldr_x = int(lm.landmark[lmPose.LEFT_SHOULDER].x * w)
          l_shldr_y = int(lm.landmark[lmPose.LEFT_SHOULDER].y * h)
          
          # Right shoulder.
          r_shldr_x = int(lm.landmark[lmPose.RIGHT_SHOULDER].x * w)
          r_shldr_y = int(lm.landmark[lmPose.RIGHT_SHOULDER].y * h)
          
          # Left ear.
          l_ear_x = int(lm.landmark[lmPose.LEFT_EAR].x * w)
          l_ear_y = int(lm.landmark[lmPose.LEFT_EAR].y * h)
          
          # Left hip.
          l_hip_x = int(lm.landmark[lmPose.LEFT_HIP].x * w)
          l_hip_y = int(lm.landmark[lmPose.LEFT_HIP].y * h)
          
          offset = findDistance(l_shldr_x, l_shldr_y, r_shldr_x, r_shldr_y)
  
          # Assist to align the camera to point at the side view of the person.
          # Offset threshold 30 is based on results obtained from analysis over 100 samples.
          if offset < 100:
              cv2.putText(image, str(int(offset)) + ' Aligned', (w - 150, 30), font, 0.9, green, 2)
          else:
              cv2.putText(image, str(int(offset)) + ' Not Aligned', (w - 150, 30), font, 0.9, red, 2)
          # Calculate angles.
          neck_inclination = findAngle(l_shldr_x, l_shldr_y, l_ear_x, l_ear_y)
          torso_inclination = findAngle(l_hip_x, l_hip_y, l_shldr_x, l_shldr_y)
          
          # Draw landmarks.
          cv2.circle(image, (l_shldr_x, l_shldr_y), 7, yellow, -1)
          cv2.circle(image, (l_ear_x, l_ear_y), 7, yellow, -1)
          
          # Let's take y - coordinate of P3 100px above x1,  for display elegance.
          # Although we are taking y = 0 while calculating angle between P1,P2,P3.
          cv2.circle(image, (l_shldr_x, l_shldr_y - 100), 7, yellow, -1)
          cv2.circle(image, (r_shldr_x, r_shldr_y), 7, pink, -1)
          #cv2.circle(image, (l_hip_x, l_hip_y), 7, yellow, -1)
          
          # Similarly, here we are taking y - coordinate 100px above x1. Note that
          # you can take any value for y, not necessarily 100 or 200 pixels.
          cv2.circle(image, (l_hip_x, l_hip_y - 100), 7, yellow, -1)
          
          # Put text, Posture and angle inclination.
          # Text string for display.
          angle_text_string = 'Neck : ' + str(int(neck_inclination)) + '  Torso : ' + str(int(torso_inclination))
          
          if neck_inclination < 70 and torso_inclination < 50 and neck_inclination>30 and torso_inclination>27:
            bad_frames = 0
            good_frames += 1
        
            # Join landmarks.
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), green, 4)
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), green, 4)
            cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), green, 4)
            cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), green, 4)
      
          else:
            good_frames = 0
            bad_frames += 1
            # Join landmarks.
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_ear_x, l_ear_y), pink, 4)
            cv2.line(image, (l_shldr_x, l_shldr_y), (l_shldr_x, l_shldr_y - 100), pink, 4)
            cv2.line(image, (l_hip_x, l_hip_y), (l_shldr_x, l_shldr_y), pink, 4)
            #cv2.line(image, (l_hip_x, l_hip_y), (l_hip_x, l_hip_y - 100), red, 4)
      
      # Calculate the time of remaining in a particular posture.
          good_time = (1 / fps) * good_frames
          bad_time =  (1 / fps) * bad_frames
              
          mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                    mp_drawing.DrawingSpec(color = (0,117, 0), thickness=2, circle_radius=4),
                                    mp_drawing.DrawingSpec(color = (0, 128, 0), thickness=2, circle_radius=2)
                                  )
          
        # save images error
          images.append(image)
          im = Image.fromarray(image)
          count+=1
          if not os.path.isdir("files"):
              os.mkdir("files")
          
          im.save(f"files/image_{count}.jpeg")
        # Display the resulting frame
          cv2.imshow('Frame',image)
    
        # Press Q on keyboard to  exit
          if cv2.waitKey(25) & 0xFF == ord('q'):
              break
    
      # Break the loop
      else: 
        break
  # When everything done, release the video capture object
  cap.release()
  
  # Closes all the frames
  cv2.destroyAllWindows()

In [127]:
get_keypoints(PATH)

In [128]:
import os
import re

def extract_number(filename):
    match_ = re.search(r'\d+', filename)
    return int(match_.group())


def create_video_from_images(images_folder_path, output_video_path):
    #image_files = sorted(os.listdir(images_folder_path))
    image_files = os.listdir(images_folder_path)
    image_files = sorted(image_files, key=extract_number)
    print(image_files)

    img = cv2.imread(os.path.join(images_folder_path, image_files[0]))
    height, width, channels = img.shape

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

    for image_file in image_files:
        img = cv2.imread(os.path.join(images_folder_path, image_file))
        video.write(img)

    cv2.destroyAllWindows()
    video.release()
    
if not os.path.isdir("output"):
        os.mkdir("output")
   
create_video_from_images('files','output/output.mp4')

['image_1.jpeg', 'image_2.jpeg', 'image_3.jpeg', 'image_4.jpeg', 'image_5.jpeg', 'image_6.jpeg', 'image_7.jpeg', 'image_8.jpeg', 'image_9.jpeg', 'image_10.jpeg', 'image_11.jpeg', 'image_12.jpeg', 'image_13.jpeg', 'image_14.jpeg', 'image_15.jpeg', 'image_16.jpeg', 'image_17.jpeg', 'image_18.jpeg', 'image_19.jpeg', 'image_20.jpeg', 'image_21.jpeg', 'image_22.jpeg', 'image_23.jpeg', 'image_24.jpeg', 'image_25.jpeg', 'image_26.jpeg', 'image_27.jpeg', 'image_28.jpeg', 'image_29.jpeg', 'image_30.jpeg', 'image_31.jpeg', 'image_32.jpeg', 'image_33.jpeg', 'image_34.jpeg', 'image_35.jpeg', 'image_36.jpeg', 'image_37.jpeg', 'image_38.jpeg', 'image_39.jpeg', 'image_40.jpeg', 'image_41.jpeg', 'image_42.jpeg', 'image_43.jpeg', 'image_44.jpeg', 'image_45.jpeg', 'image_46.jpeg', 'image_47.jpeg', 'image_48.jpeg', 'image_49.jpeg', 'image_50.jpeg', 'image_51.jpeg', 'image_52.jpeg', 'image_53.jpeg', 'image_54.jpeg', 'image_55.jpeg', 'image_56.jpeg', 'image_57.jpeg', 'image_58.jpeg', 'image_59.jpeg', 'imag

DRAW LANDMARKS